# 代码随想录算法训练营第二天｜977.有序数组的平方 ，209.长度最小的子数组 ，59.螺旋矩阵II

## 977.有序数组的平方 

题目建议： 本题关键在于理解双指针思想 

题目链接：https://leetcode.com/problems/squares-of-a-sorted-array/  
文章讲解：https://programmercarl.com/0977.%E6%9C%89%E5%BA%8F%E6%95%B0%E7%BB%84%E7%9A%84%E5%B9%B3%E6%96%B9.html  
视频讲解： https://www.bilibili.com/video/BV1QB4y1D7ep 


## 209.长度最小的子数组

题目建议： 本题关键在于理解滑动窗口 

题目链接：https://leetcode.com/problems/minimum-size-subarray-sum/  
文章讲解：https://programmercarl.com/0209.%E9%95%BF%E5%BA%A6%E6%9C%80%E5%B0%8F%E7%9A%84%E5%AD%90%E6%95%B0%E7%BB%84.html  
视频讲解：https://www.bilibili.com/video/BV1tZ4y1q7XE


## 59.螺旋矩阵II

题目建议：  本题关键还是在转圈的逻辑，在二分搜索中提到的区间定义，在这里又用上了。 

题目链接：https://leetcode.com/problems/spiral-matrix-ii/  
文章讲解：https://programmercarl.com/0059.%E8%9E%BA%E6%97%8B%E7%9F%A9%E9%98%B5II.html  
视频讲解：https://www.bilibili.com/video/BV1SL4y1N7mV/


## 总结 

题目建议：希望大家 也做一个自己 对数组专题的总结

文章链接：https://programmercarl.com/%E6%95%B0%E7%BB%84%E6%80%BB%E7%BB%93%E7%AF%87.html 
